In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import bson
from tqdm import tqdm_notebook
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cdiscount-image-classification-challenge/sample_submission.csv
/kaggle/input/cdiscount-image-classification-challenge/test.bson
/kaggle/input/cdiscount-image-classification-challenge/train_example.bson
/kaggle/input/cdiscount-image-classification-challenge/category_names.csv
/kaggle/input/cdiscount-image-classification-challenge/train.bson


In [ ]:
prod_id = []
prod_category = []
prod_num_imgs = []

num_dicts = 7069896 # according to data page

# This will take about 02m15s to complete
with open('../input/cdiscount-image-classification-challenge/train.bson', 'rb') as f, tqdm_notebook(total=num_dicts) as bar:
        
    data = bson.decode_file_iter(f)

    for c, d in enumerate(data):
        bar.update()
        prod_id.append(d['_id'])
        prod_category.append(d['category_id'])
        prod_num_imgs.append(len(d['imgs']))

In [ ]:
df_dict = {
    'prod_id': prod_id,
    'category_id': prod_category,
    'num_imgs': prod_num_imgs
}
df = pd.DataFrame(df_dict, index=prod_id)

In [ ]:
df.head()

In [ ]:
df.to_csv('bson.csv')

In [ ]:
categories = pd.read_csv('../input/cdiscount-image-classification-challenge/category_names.csv', index_col='category_id')

In [ ]:
categories.head()

In [ ]:
df1 = df.merge(categories, on = 'category_id')

In [ ]:
df1.head()

In [ ]:
df.num_imgs.sum()

In [ ]:
df_catimg = df1.groupby(['category_id'], as_index=False)['num_imgs'].sum().sort_values(by = 'num_imgs', ascending = True)
df_catimg2 = df_catimg.merge(categories, on = 'category_id')

In [ ]:
df_catimg2.to_csv('cat_img.csv')

In [2]:
#Function to save images from each category
#When catergory_id is given as input, the functions saves all images associated to it.
#The image is stored as the product_id-no.jpg (no is number of images associated with the product_id)
count = 0
def bson2img(categoryid):
    with open('../input/cdiscount-image-classification-challenge/train.bson', 'rb') as f:    
        data = bson.decode_file_iter(f)
        count = 0
        for c, d in enumerate(data):
            if d['category_id'] == categoryid:
                for e, pic in enumerate(d['imgs']):
                    if count ==3:
                        break
                    id1 = d['_id']
                    fname = '{}-{}.jpg'.format(id1, e)
                    print(fname)
                    with open(fname, 'wb') as f:
                        f.write(pic['picture'])
                        count += 1
            